In [2]:
print("Notebook is working")

Notebook is working


## Imports

In [3]:
import requests
from datetime import datetime, time, date as dt_date

## Stadium and game information

In [37]:
# Phillies (Citizens Bank Park)
stadium_name = "Citizens Bank Park"
latitude= 39.9057
longitude= -75.1665
date = dt_date.today().isoformat() #returns YYYY-MM-DD for today

In [4]:
# Braves (Truist Park)
stadium_name = "Truist Park"
latitude = 33.8908
longitude = -84.4678
team_id = 144
date = dt_date.today().isoformat()

In [32]:
def get_weather_data(lat, lon, date):
    weather_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": date,
        "end_date": date,
        "hourly": "winddirection_10m,windspeed_10m",
        "timezone": "America/New_York"
    }
    response = requests.get(weather_url, params=params)
    return response.json()

In [33]:
# Get the weather data for today's date and Citizens Bank Park
weather_data = get_weather_data(latitude, longitude, date)

# Show the top-level keys of the response
print(weather_data.keys())


dict_keys(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'hourly_units', 'hourly'])


In [34]:
def filter_for_game_time(weather_data, start_hour=19, end_hour=22):
    hourly = weather_data['hourly']
    times = hourly['time']
    directions = hourly['winddirection_10m']
    speeds_kmh = hourly['windspeed_10m']  # Original API values

    game_start = time(start_hour, 0)
    game_end = time(end_hour, 0)

    filtered = []

    for i, t_str in enumerate(times):
        dt = datetime.strptime(t_str, '%Y-%m-%dT%H:%M')
        if game_start <= dt.time() <= game_end:
            filtered.append({
                "time": t_str,
                "direction_deg": directions[i],
                "speed_kmh": speeds_kmh[i],
                "speed_mph": round(speeds_kmh[i] * 0.621371, 2)  # Converted on the fly
            })

    return filtered


In [35]:
game_time_data = filter_for_game_time(weather_data)

for entry in game_time_data:
    print(f"Time: {entry['time']}, Direction: {entry['direction_deg']}°, "
          f"Speed: {entry['speed_kmh']} km/h ({entry['speed_mph']} mph)")


Time: 2025-07-03T19:00, Direction: 257°, Speed: 19.2 km/h (11.93 mph)
Time: 2025-07-03T20:00, Direction: 218°, Speed: 15.1 km/h (9.38 mph)
Time: 2025-07-03T21:00, Direction: 349°, Speed: 19.4 km/h (12.05 mph)
Time: 2025-07-03T22:00, Direction: 11°, Speed: 7.7 km/h (4.78 mph)


In [36]:
# 1. Call MLB schedule API for the Phillies (team ID 143) on today's date
mlb_url = "https://statsapi.mlb.com/api/v1/schedule"
mlb_params = {
    "sportId": 1,
    "date": date,     # today's date variable you've set
    "teamId": 143     # Phillies team ID
}
mlb_resp = requests.get(mlb_url, params=mlb_params)
mlb_data = mlb_resp.json()


games = mlb_data.get("dates", [])
if games:
    game = games[0]["games"][0]  # first Phillies game of the day
    utc_start = game['gameDate']  # e.g. "2025-06-30T23:05:00Z"
    from datetime import datetime, timezone
    dt_utc = datetime.fromisoformat(utc_start.replace("Z", "+00:00"))
    dt_local = dt_utc.astimezone(timezone.utc).astimezone(tz=None)
    print("Phillies game local start time:", dt_local)
else:
    print("No Phillies game today.")

No Phillies game today.


In [39]:
# 1. Call MLB schedule API for the Braves (team ID 144) on today's date
mlb_url = "https://statsapi.mlb.com/api/v1/schedule"
mlb_params = {
    "sportId": 1,
    "date": date,     # today's date variable you've set
    "teamId": 144     # Braves team ID
}
mlb_resp = requests.get(mlb_url, params=mlb_params)
mlb_data = mlb_resp.json()

games = mlb_data.get("dates", [])
if games:
    game = games[0]["games"][0]  # first Braves game of the day
    utc_start = game['gameDate']  # e.g. "2025-07-01T23:20:00Z"
    from datetime import datetime, timezone
    dt_utc = datetime.fromisoformat(utc_start.replace("Z", "+00:00"))
    dt_local = dt_utc.astimezone(timezone.utc).astimezone(tz=None)
    game_hour = dt_local.hour
    print("Braves game local start time:", dt_local)
else:
    game_hour = None
    print("No Braves game today.")


Braves game local start time: 2025-07-03 19:15:00-04:00
